In [ ]:
!pip install utilsforecast uni2ts

In [ ]:
!pip install --upgrade torch torchvision

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate
from utilsforecast.losses import *

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split

from uni2ts.model.moirai2 import Moirai2Forecast, Moirai2Module

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/marcopeix/FoundationModelsForTimeSeriesForecasting/refs/heads/main/data/Walmart.csv"

df = pd.read_csv(DATA_URL)
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y')

df.head()

## Zero-shot forecasting

In [ ]:
HORIZON = 13

In [ ]:
gluon_df = df.copy()
gluon_df = gluon_df.set_index('Date')
ds = PandasDataset.from_long_dataframe(gluon_df, target='Weekly_Sales', item_id='Store')

In [ ]:
model = Moirai2Forecast(
    module=Moirai2Module.from_pretrained(
        f"Salesforce/moirai-2.0-R-small",
    ),
    prediction_length=HORIZON,
    context_length=len(df.query('Store == 1')),
    target_dim=1,
    feat_dynamic_real_dim=0,
    past_feat_dynamic_real_dim=0,
)

In [ ]:
predictor = model.create_predictor(batch_size=32)
forecasts = predictor.predict(ds)
forecasts = list(forecasts)

In [ ]:
forecasts[0]

In [ ]:
def create_fcsts_df(forecasts, multivariate=False):
    all_forecast_data = []
    for forecast_obj in forecasts:
        # Check if the forecast object has 'quantile' method (e.g., QuantileForecast)
        if hasattr(forecast_obj, 'quantile'):
            q10 = forecast_obj.quantile('0.1')
            q50 = forecast_obj.quantile('0.5')
            q90 = forecast_obj.quantile('0.9')
        # Otherwise, assume it's a SampleForecast and calculate quantiles from samples
        elif hasattr(forecast_obj, 'samples'):
            q10 = np.quantile(forecast_obj.samples, 0.1, axis=0)
            q50 = np.quantile(forecast_obj.samples, 0.5, axis=0)
            q90 = np.quantile(forecast_obj.samples, 0.9, axis=0)
        else:
            raise ValueError("Unsupported forecast object type. It must have either 'quantile' method or 'samples' attribute.")

        start_date = forecast_obj.start_date.to_timestamp()
        frequency = forecast_obj.start_date.freqstr

        if multivariate:
            # For multivariate forecasts, item_id is None, and samples are (prediction_length, num_series)
            num_series = q50.shape[1] if q50.ndim > 1 else 1
            original_store_ids = sorted(df['Store'].unique())

            for i in range(num_series):
                current_item_id = original_store_ids[i]
                dates = pd.date_range(start=start_date, periods=len(q50[:, i]), freq=frequency)
                temp_df = pd.DataFrame({
                    'Store': current_item_id,
                    'Date': dates,
                    'Moirai2': q50[:, i],
                    'Moirai2-lo-80': q10[:, i],
                    'Moirai2-hi-80': q90[:, i],
                })
                all_forecast_data.append(temp_df)
        else:
            # For univariate forecasts, item_id is present
            item_id = forecast_obj.item_id
            dates = pd.date_range(start=start_date, periods=len(q50), freq=frequency)
            temp_df = pd.DataFrame({
                'Store': item_id,
                'Date': dates,
                'Moirai2': q50,
                'Moirai2-lo-80': q10,
                'Moirai2-hi-80': q90,
            })
            all_forecast_data.append(temp_df)

    final_df = pd.concat(all_forecast_data, ignore_index=True)
    return final_df

In [ ]:
fcsts_df = create_fcsts_df(forecasts)
fcsts_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=fcsts_df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

## Cross-validation

In [ ]:
train, test_template = split(ds, offset=-(3*HORIZON))

test_data = test_template.generate_instances(
    prediction_length=HORIZON,
    windows=3,
    distance=HORIZON
)

predictor = model.create_predictor(batch_size=32)
cv_forecasts = predictor.predict(test_data.input)
cv_forecasts = list(cv_forecasts)

In [ ]:
len(cv_forecasts)

In [ ]:
cv_fcsts_df = create_fcsts_df(cv_forecasts)
cv_fcsts_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=cv_fcsts_df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
test_df = cv_fcsts_df.merge(df[["Store", "Date", "Weekly_Sales"]], on=['Store', 'Date'], how='left')
test_df.head()

In [ ]:
eval_df = evaluate(
    test_df,
    metrics=[mae, smape],
    models=['Moirai2'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df

## Forecasting with covariates

In [ ]:
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule

In [ ]:
exog_ds = PandasDataset.from_long_dataframe(
    gluon_df,
    target='Weekly_Sales',
    item_id='Store',
    past_feat_dynamic_real=["Temperature", "Fuel_Price", "CPI", "Unemployment"],
    feat_dynamic_real=["Holiday_Flag"]
)

In [ ]:
train, test_template = split(exog_ds, offset=-(3*HORIZON))

test_data = test_template.generate_instances(
    prediction_length=HORIZON,
    windows=3,
    distance=HORIZON
)

model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-small"),
    prediction_length=HORIZON,
    context_length=len(df.query('Store == 1')),
    patch_size="auto",
    num_samples=100,
    target_dim=1,
    feat_dynamic_real_dim=exog_ds.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=exog_ds.num_past_feat_dynamic_real,
)

predictor = model.create_predictor(batch_size=32)
exog_cv_forecasts = predictor.predict(test_data.input)
exog_cv_forecasts = list(exog_cv_forecasts)

In [ ]:
exog_cv_fcsts_df = create_fcsts_df(exog_cv_forecasts)
exog_cv_fcsts_df["Store"] = exog_cv_fcsts_df["Store"].astype(int)
exog_cv_fcsts_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=exog_cv_fcsts_df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
exog_test_df = exog_cv_fcsts_df.merge(df[["Store", "Date", "Weekly_Sales"]], on=['Store', 'Date'], how='left')

eval_df = evaluate(
    exog_test_df,
    metrics=[mae, smape],
    models=['Moirai2'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df

## Multivariate forecasting

In [ ]:
from gluonts.dataset.multivariate_grouper import MultivariateGrouper

In [ ]:
num_stores = df['Store'].nunique()

# Initialize the MultivariateGrouper
grouper = MultivariateGrouper(
    max_target_dim=num_stores,
)

multivariate_ds = grouper(ds)

In [ ]:
train, test_template = split(multivariate_ds, offset=-(3*HORIZON))

test_data = test_template.generate_instances(
    prediction_length=HORIZON,
    windows=3,
    distance=HORIZON
)

model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(f"Salesforce/moirai-1.0-R-small"),
    prediction_length=HORIZON,
    context_length=len(df.query('Store == 1')),
    patch_size="auto",
    num_samples=100,
    target_dim=num_stores,
    feat_dynamic_real_dim=0,
    past_feat_dynamic_real_dim=0,
)

predictor = model.create_predictor(batch_size=32)
mv_cv_forecasts = predictor.predict(test_data.input)
mv_cv_forecasts = list(mv_cv_forecasts)

In [ ]:
mv_cv_fcsts_df = create_fcsts_df(mv_cv_forecasts, multivariate=True)
mv_cv_fcsts_df["Store"] = mv_cv_fcsts_df["Store"].astype(int)
mv_cv_fcsts_df.head()

In [ ]:
plot_series(
    df=df,
    forecasts_df=mv_cv_fcsts_df,
    id_col="Store",
    time_col="Date",
    target_col="Weekly_Sales",
    level=[80],
    max_ids=6,
)

In [ ]:
mv_test_df = mv_cv_fcsts_df.merge(df[["Store", "Date", "Weekly_Sales"]], on=['Store', 'Date'], how='left')

eval_df = evaluate(
    mv_test_df,
    metrics=[mae, smape],
    models=['Moirai2'],
    target_col='Weekly_Sales',
    id_col='Store',
    time_col="Date",
    agg_fn="mean"
)
eval_df